In [1]:
import numpy as np
from sklearn.manifold import TSNE
import json
import glob
from collections import defaultdict
import matplotlib.pyplot as plt

In [4]:
full_embedding_files = glob.glob('../preprocess/data/full_text_embedding/*.json')
all_embeddings = []
for file in full_embedding_files:
    article_w_embedding = json.load(open(file))
    all_embeddings.append(article_w_embedding)
save_json(all_embeddings, '../preprocess/data/full/full_embedding.json')


In [6]:
full_embeddings = json.load(open('../preprocess/data/full/full_embedding.json'))
summary_embeddings = json.load(open('../preprocess/data/summary/summary_embedding.json'))
full_embeddings_dict = {full['id']: full for full in full_embeddings}
summary_embeddings_dict = {summary['id']: summary for summary in summary_embeddings}

In [11]:
def generate_cluster_dict(embeddings_dict, cluster_file, level):
    clusters_dict = defaultdict(list)
    partition = json.load(open(cluster_file))[level]
    for node_id, cluster_label in partition.items():
        node_data = embeddings_dict[node_id]
        clusters_dict[cluster_label].append(node_data)
    return clusters_dict
full_cluster_dict = generate_cluster_dict(full_embeddings_dict, '../preprocess/data/full/full_partitions.json', level=4)
summary_cluster_dict = generate_cluster_dict(summary_embeddings_dict, '../preprocess/data/summary/summary_partitions.json', level=4)

In [3]:
def plot(XY):
    plt.figure(figsize=(8, 6))
    plt.scatter(XY[:, 0], XY[:, 1])
    plt.title('Scatterplot')
    plt.xlabel('Dimension 1')
    plt.ylabel('Dimension 2')
    plt.show()
def min_max_normalize(data):
    min_vals = np.min(data, axis=0)
    max_vals = np.max(data, axis=0)
    
    normalized_data = (data - min_vals) / (max_vals - min_vals)
    
    return normalized_data
def save_json(data, filepath=r'new_data.json'):
    with open(filepath, 'w', encoding='utf-8') as fp:
        json.dump(data, fp, indent=4)

In [25]:
from sklearn.manifold import MDS, SpectralEmbedding, TSNE, LocallyLinearEmbedding
import umap
reducer = umap.UMAP()

# save_json(chunks, r'../data/result/chunk_coordinates.json')
node_coordinates = {}
for topic, nodes in summary_cluster_dict.items():
    print(topic)
    print(len(nodes))
    embeddings = np.array([node['embedding'] for node in nodes])
    print(embeddings)
    node_ids = [node['id'] for node in nodes]
    if len(nodes) == 1:
        XY = np.array([[0.5, 0.5]])
    else:
        XY = TSNE(n_components=2, learning_rate='auto', init='random', perplexity=min(len(nodes)-1, 50), metric='cosine').fit_transform(embeddings)
        # XY = MDS(n_components=2).fit_transform(embeddings)
        # XY = reducer.fit_transform(embeddings)
        # plot(XY)
        XY = min_max_normalize(XY)
    for node_id, coordinate in zip(node_ids, XY):
        node_coordinates[node_id] = coordinate.tolist()
save_json(node_coordinates, r'../preprocess/data/summary/node_coordinates.json')

1
124
[[-0.00203811 -0.00078149  0.01793665 ... -0.02254709 -0.00635877
  -0.00197983]
 [-0.00452415 -0.00476347  0.00698948 ... -0.00414058 -0.02224044
  -0.00938268]
 [-0.00509154 -0.00672749  0.01039394 ... -0.02927718 -0.02222998
  -0.01313528]
 ...
 [ 0.0156426  -0.0208965  -0.00853594 ...  0.00135318 -0.01865995
  -0.0091447 ]
 [-0.00956658 -0.01405051 -0.00010572 ... -0.00564482 -0.01519477
  -0.02335098]
 [-0.02237016 -0.00771654  0.00893117 ... -0.00030711 -0.01089279
  -0.0097366 ]]
0
3375
[[-0.02259278 -0.01817918  0.00364324 ... -0.00635464 -0.00170892
  -0.00043606]
 [-0.02015544  0.00288459  0.01567349 ... -0.00653618 -0.00867711
  -0.00793012]
 [-0.01677093 -0.03082442 -0.00095744 ... -0.01595836  0.00624081
  -0.01898219]
 ...
 [-0.03235813  0.00424159  0.0205486  ... -0.01631028  0.01355472
  -0.01774055]
 [ 0.01712982  0.00550323 -0.00530193 ... -0.00270778 -0.01244801
   0.00784413]
 [-0.02430049 -0.01602358 -0.00404469 ...  0.01655382 -0.00034999
  -0.01218904]]
6
3

In [27]:
save_json(chunk_coordinates, r'../data/result/chunk_coordinates.json')